<a href="https://colab.research.google.com/github/tqmbow/Sentinel1Clearfelling/blob/master/GEE_Export_S1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install GEE and initialise

In [0]:
! pip install earthengine-api

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 61kB 22.5MB/s 
    100% |████████████████████████████████| 2.3MB 7.7MB/s 
    100% |████████████████████████████████| 102kB 25.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/23/66/14/73fd4f4385532312ca3d26df889b37339b9330bcf4fd1d8e40
Successfully built earthengine-api


In [0]:
! earthengine authenticate

Opening the following address in a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

Please authorize access to your Earth Engine account, and paste the generated code below. If the web browser does not start, please manually browse the URL above.

Please enter authorization code: 4/IQFNBRDqeFUMHkuuKHrYsW6_-E8nbSSZecUlG0IQwiN1QXKZxK5Ma2A

Successfully saved authorization token.


In [0]:
import ee
ee.Initialize()
from IPython.display import Image,display

In [0]:
#Import extent
extent = ee.FeatureCollection('users/tomw_ee/rsRegion4Scotland')

In [0]:
#Import S1 image collection for month in question
dateFrom = '2016-05-15'
dateTo = '2016-08-15'
s1c = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterBounds(extent).filterDate(dateFrom, dateTo).filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV', 'VH']))

In [0]:
def toDB(img):
  outimg = img.select(['VV','VH'])
  return ee.Image(outimg).log10().multiply(10.0).copyProperties(img)

s1c = s1c.select(['VV','VH']).map(toDB)


In [0]:
#Optional - mask image edges
def maskEdge(img):
    mask = img.select(0).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1,1), 100)
    return img.updateMask(mask.select(0))
s1c = s1c.map(maskEdge)

In [0]:
#make median composite of monthly values
img = s1c.median()

In [0]:
#Create Ascending / Descending Composites
imgasc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).median().select(['VV', 'VH'],['VVasc','VHasc'])
imgdesc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).median().select(['VV', 'VH'],['VVdesc','VHdesc'])

In [0]:
#Create ratio bands
img = img.addBands(img.select('VV').subtract(img.select('VH')).rename('Ratio'))
imgasc = imgasc.addBands(imgasc.select('VVasc').subtract(imgasc.select('VHasc')).rename('Ratioasc'))
imgdesc = imgdesc.addBands(imgdesc.select('VVdesc').subtract(imgdesc.select('VHdesc')).rename('Ratiodesc'))

In [0]:
#Add Percentiles
s10pasc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
.reduce(ee.Reducer.percentile([10])).select(['VV_p10','VH_p10'],['VVasc_p10','VHasc_p10'])

s10pasc = s10pasc.addBands(s10pasc.select('VVasc_p10').subtract(s10pasc.select('VHasc_p10')).rename('Ratioasc_p10'))

s10pdesc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
.reduce(ee.Reducer.percentile([10])).select(['VV_p10','VH_p10'],['VVdesc_p10','VHdesc_p10'])

s10pdesc = s10pdesc.addBands(s10pdesc.select('VVdesc_p10').subtract(s10pdesc.select('VHdesc_p10')).rename('Ratiodesc_p10'))

#######

s90pasc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
.reduce(ee.Reducer.percentile([90])).select(['VV_p90','VH_p90'],['VVasc_p90','VHasc_p90'])

s90pasc = s90pasc.addBands(s90pasc.select('VVasc_p90').subtract(s90pasc.select('VHasc_p90')).rename('Ratioasc_p90'))

s90pdesc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
.reduce(ee.Reducer.percentile([90])).select(['VV_p90','VH_p90'],['VVdesc_p90','VHdesc_p90'])

s90pdesc = s90pdesc.addBands(s90pdesc.select('VVdesc_p90').subtract(s90pdesc.select('VHdesc_p90')).rename('Ratiodesc_p90'))


In [0]:
srtm = ee.Image('USGS/SRTMGL1_003')
slope = ee.Terrain.slope(srtm).select('slope')
aspect = ee.Terrain.aspect(srtm).select('aspect')

In [0]:
#Create slope and aspect layers
import math

#Local incidence angle uses aspect calculated from SRTM
im1inc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first().select('angle')
s1_azimuth = ee.Terrain.aspect(im1inc).reduceRegion(ee.Reducer.mean(), im1inc.get('system:footprint'), 1000).get('aspect')
slope_projected = slope.multiply(ee.Image.constant(s1_azimuth).subtract(aspect).multiply(math.pi/180).cos())
liaAsc = im1inc.subtract(ee.Image.constant(90).subtract(ee.Image.constant(90).subtract(slope_projected))).abs()
liaAsc = liaAsc.rename('liaAsc')

im2inc = s1c.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first().select('angle')
s1_azimuth2 = ee.Terrain.aspect(im2inc).reduceRegion(ee.Reducer.mean(), im2inc.get('system:footprint'), 1000).get('aspect')
slope_projected2 = slope.multiply(ee.Image.constant(s1_azimuth).subtract(aspect).multiply(math.pi/180).cos())
liaDesc = im2inc.subtract(ee.Image.constant(90).subtract(ee.Image.constant(90).subtract(slope_projected))).abs()
liaDesc = liaDesc.rename('liaDesc')
print(s1_azimuth.getInfo())

None


In [0]:
#Add all bands to composite image
img = img.addBands(imgdesc).addBands(imgasc).addBands(s10pdesc).addBands(s10pasc).addBands(s90pasc).addBands(s90pdesc)#.addBands(slope).addBands(aspect)

In [0]:
#Select bands to use
img = img.select(['VVdesc','VHdesc','VVasc','VHasc','VVdesc_p10','VHdesc_p10','VVasc_p10','VHasc_p10','VVasc_p90','VHasc_p90','VVdesc_p90','VHdesc_p90'])#['VVdesc_p10','VHdesc_p10','Ratiodesc_p10','VVasc_p10','VHasc_p10','Ratioasc_p10', 'slope','aspect']) 
                 #['VVasc','VHasc','Ratioasc','liaAsc'])#
#img= img.select(['VVdesc','VHdesc','Ratiodesc','VVasc','VHasc','Ratioasc'])


In [0]:
#View Image
url = img.getThumbUrl({'bands': ['VVdesc','VHdesc','Ratiodesc'], 'min': [-15,-20,0], 'max': [-5,-10,15], 'region':extent.geometry().bounds().getInfo()['coordinates']})
Image(url=url)

In [0]:
def export_image(img, output_file_name):
    task=ee.batch.Export.image.toDrive(
            image=img,
            folder = "colab_data",
            description = output_file_name,
            region = extent.geometry().bounds().getInfo()['coordinates'],
            scale= 10,
            crs= 'EPSG:27700',
            maxPixels =10000000000)
    task.start()

In [0]:
export_image(img.float(), 'rsRegion4_3month16')

In [0]:
visImg = img.visualize(bands=['VVasc','VHasc','Ratioasc'], min=[-15,-20,0],max=[-5,-10,15])
#visImg2 = img.visualize(bands=['VVdesc_p10','VHdesc_p10','Ratiodesc_p10'],min=[-15,-20,0],max=[-5,-10,15])
#visImg = visImg.addBands(visImg2)

In [0]:
export_image(visImg.int8(), 'rsRegion3AscRGB')

In [0]:
!earthengine task list

NSJ2C4TV3O7MYKS3F7WDPQO4  Export.image  rsRegion4_3month16                          COMPLETED  ---
LWBIU4VK2OQ5EQ3OZOQ2AW5O  Export.image  rsRegion4_3month16                          COMPLETED  ---
EXOJVLWNYPI5MPZBOJXAC5AE  Export.image  rsRegion4_3month16                          COMPLETED  ---
OHS3ZNU3UVPQMKR473SE63R5  Export.image  rsRegion4_3month                            COMPLETED  ---
IEUW7GCWS4HV3NEQX6WTRQCT  Export.image  rsRegion4_3monthAllPercentile               COMPLETED  ---
QJWIJCUGLJ4PJFNTUU5ECP2R  Upload        Asset ingestion: users/tomw_ee/RSRegion4..  COMPLETED  ---
JITDTODGVIZP6BV5KZ2SGZ5Z  Export.image  rsRegion2_3monthAllPercentile               COMPLETED  ---
Y56MF7YMDH5D2S6BFVLEBIPA  Export.image  rsRegion2_3monthAllPercentile               COMPLETED  ---
